In [9]:
using Plots, LinearAlgebra, Printf
gr(format="png")

Plots.GRBackend()

Here I give an example of simple Bayesian inference using the following situation:

_Suppose you have three six-sided dice (3d6, in Dungeons and Dragons jargon) and one 16-sided one (d16). These each have sixteen possible results: either 3 through 18 or 1 through 16. Suppose if you roll the 3d6 you simply record the sum, whereas if you roll the d16, you add two to it (d16 + 2) giving you the same possible set of results_

_Someone presents you a series of such die rolls._ **Can you determine which set of dice was used?**

This presents a nice clean example of Bayesian inference. What are the probablilities for each set of dice?

For the 3d6, we have a multinomial distribution, which is a pretty good approximation to a Gaussian. It is symmetric about 10 and 11, so we can write it in a short table:

|3d6 Roll | Probability | 
|---------|-------------|
| 3 or 18 | 1/216       |
| 4 or 17 | 3/216       |
| 5 or 16 | 6/216       |
| 6 or 15 | 10/216      |
| 7 or 14 | ...         |
| 8 or 13 | (see below) |
| 9 or 12 |             |
|10 or 11 |             |

For the d16, the discrete probability distribution is uniform:

|d16+2 Roll | Probability |
|-----------|-------------|
| 3 through 18 | 0.0625 each  |



Since this is such a short set of values, we can make a dictionary instead of an array or function:

In [8]:
vals3 = [1, 3, 6, 10, 15, 21, 25, 27, 27, 25, 21, 15, 10, 6, 3, 1]/216
k3 = collect(3:18)
r3d6 = Dict(zip(k3, vals3))

Dict{Int64,Float64} with 16 entries:
  16 => 0.0277778
  11 => 0.125
  7  => 0.0694444
  9  => 0.115741
  10 => 0.125
  17 => 0.0138889
  8  => 0.0972222
  6  => 0.0462963
  4  => 0.0138889
  3  => 0.00462963
  5  => 0.0277778
  13 => 0.0972222
  14 => 0.0694444
  15 => 0.0462963
  12 => 0.115741
  18 => 0.00462963

In [12]:
for x in 3:18
    @printf("%4d | %7.5f | %7.5f \n", x, r3d6[x], 10*log10(r3d6[x]))
end

   3 | 0.00463 | -23.34454 
   4 | 0.01389 | -18.57332 
   5 | 0.02778 | -15.56303 
   6 | 0.04630 | -13.34454 
   7 | 0.06944 | -11.58362 
   8 | 0.09722 | -10.12234 
   9 | 0.11574 | -9.36514 
  10 | 0.12500 | -9.03090 
  11 | 0.12500 | -9.03090 
  12 | 0.11574 | -9.36514 
  13 | 0.09722 | -10.12234 
  14 | 0.06944 | -11.58362 
  15 | 0.04630 | -13.34454 
  16 | 0.02778 | -15.56303 
  17 | 0.01389 | -18.57332 
  18 | 0.00463 | -23.34454 


In [18]:
# Now let's make the log-odds of 3d6 to d16+2:
# units are deciban
vals_lo = [10*log10(r3d6[x]) + 10*log10(16) for x in 3:18]
log_odds = Dict(zip(k3, vals_lo))

Dict{Int64,Float64} with 16 entries:
  16 => -3.52183
  11 => 3.0103
  7  => 0.457575
  9  => 2.67606
  10 => 3.0103
  17 => -6.53213
  8  => 1.91886
  6  => -1.30334
  4  => -6.53213
  3  => -11.3033
  5  => -3.52183
  13 => 1.91886
  14 => 0.457575
  15 => -1.30334
  12 => 2.67606
  18 => -11.3033

In [21]:
for x in 3:18
    @printf("%4d |%5.1f \n", x, log_odds[x])
end

   3 |-11.3 
   4 | -6.5 
   5 | -3.5 
   6 | -1.3 
   7 |  0.5 
   8 |  1.9 
   9 |  2.7 
  10 |  3.0 
  11 |  3.0 
  12 |  2.7 
  13 |  1.9 
  14 |  0.5 
  15 | -1.3 
  16 | -3.5 
  17 | -6.5 
  18 |-11.3 


In [102]:
function roll3d6()
    return rand(1:6) + rand(1:6) + rand(1:6)
end

function rolld16()
    return rand(3:18)
end

rolld16 (generic function with 1 method)

In [158]:
slen = 16
sample = [roll3d6() for i in 1:slen];

In [159]:
# do the inference
prior = 0.0 # assume either equally likely
for s in 1:slen
    increment = log_odds[sample[s]]
    posterior = prior + increment
    @printf("% 3d %3d %5.1f %5.1f \n",s, sample[s], increment, posterior)
    prior = posterior
end
# this could all be done with a cumulative sum, but this shows the history if one
# assumes the data come in one at a time:

  1  14   0.5   0.5 
  2   9   2.7   3.1 
  3   8   1.9   5.1 
  4  13   1.9   7.0 
  5  17  -6.5   0.4 
  6  12   2.7   3.1 
  7  13   1.9   5.0 
  8   8   1.9   7.0 
  9   6  -1.3   5.6 
 10  15  -1.3   4.3 
 11  16  -3.5   0.8 
 12   7   0.5   1.3 
 13  10   3.0   4.3 
 14  12   2.7   7.0 
 15  13   1.9   8.9 
 16   7   0.5   9.3 


In [160]:
@printf("log odds of being a 3d6 sample: %6.2f deciban\n", prior)
        # final prior going out of loop is same as last posterior
        # careful of loop scope rules.
odda = 10^(abs(prior)/10) #odds > 1
odds = 10^((prior)/10)

oddr = rationalize(odda, tol=0.01)
if prior < 0.0
    @printf("odds AGAINST ")
else
    @printf("odds FOR ")
end
@printf("being a 3d6 sample: %3d to %3d\n", numerator(oddr), denominator(oddr))
p = odds/(odds + 1)
@printf("probability of being a 3d6 sample: %8.6f", p)


log odds of being a 3d6 sample:   9.34 deciban
odds FOR being a 3d6 sample:  43 to   5
probability of being a 3d6 sample: 0.895831

In [162]:
43/48

0.8958333333333334

0.9999995566210425